# Simple Reinforcement Learning in Tensorflow Part 2-b: 
## Vanilla Policy Gradient Agent

THIS FILE COMES FROM THE BLOG THOUGH I HAVE TWEAKED IT A BIT SO WE CAN ACTUALLY SEE HOW THE BOT PERFORMS

This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the CartPole problem. For more information, see this [Medium post](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724#.mtwpvfi8b). This implementation is generalizable to more than two actions.

For more Reinforcement Learning algorithms, including DQN and Model-based learning in Tensorflow, see my Github repo, [DeepRL-Agents](https://github.com/awjuliani/DeepRL-Agents). 

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

try:
    xrange = xrange
except:
    xrange = range

In [2]:
env = gym.make('CartPole-v0')

[2017-11-21 23:51:22,512] Making new env: CartPole-v0


### The Policy-Based Agent

In [3]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [4]:
class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size, biases_initializer=None, activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden, a_size, activation_fn=tf.nn.softmax, biases_initializer=None)
        self.chosen_action = tf.argmax(self.output,1)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)

        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

### Training the Agent

In [6]:
tf.reset_default_graph() #Clear the Tensorflow graph.

myAgent = agent(lr=1e-3,s_size=4,a_size=2,h_size=8) #Load the agent.

total_episodes = 5000 #Set total number of episodes to train agent on.
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

# Launch the tensorflow graph
sess = tf.Session()
sess.run(init)
i = 0
total_reward = []
total_lenght = []

gradBuffer = sess.run(tf.trainable_variables())
for ix,grad in enumerate(gradBuffer):
    gradBuffer[ix] = grad * 0

while i < total_episodes:
    s = env.reset()
    running_reward = 0
    ep_history = []
    for j in range(max_ep):
        #Probabilistically pick an action given our network outputs.
        a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]})
        a = np.random.choice(a_dist[0],p=a_dist[0])
        a = np.argmax(a_dist == a)

        s1,r,d,_ = env.step(a) #Get our reward for taking an action given a bandit.
        ep_history.append([s,a,r,s1])
        s = s1
        running_reward += r
        if d == True:
            print ("Got to step: {0}".format(j))
            #Update the network.
            ep_history = np.array(ep_history)
            ep_history[:,2] = discount_rewards(ep_history[:,2])
            feed_dict={myAgent.reward_holder:ep_history[:,2],
                    myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
            grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
            for idx,grad in enumerate(grads):
                gradBuffer[idx] += grad

            if i % update_frequency == 0 and i != 0:
                feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0

            total_reward.append(running_reward)
            total_lenght.append(j)
            break


        #Update our running tally of scores.
    if i % 100 == 0:
        print(np.mean(total_reward[-100:]))
    i += 1

38.0
42.27
44.41
55.89
71.03
98.97
141.35
139.81
132.76
160.29
177.3
176.53
176.98
185.05
173.73
188.36
192.1
194.75
198.66
195.82
189.73
194.26
198.03
197.94
197.37
195.2
197.9
195.49
197.19
195.75
194.12
198.28
198.19
195.34
175.77
182.28
192.22
187.41
185.01
157.82
161.13
161.32
168.05
190.21
182.47
183.69
182.13
180.23
192.42
198.76
195.43
198.6
197.53
200.0
195.78
185.62
169.86
187.31
193.43
189.79
189.79
179.16
189.67
187.32
184.1
173.18
141.15
130.98
135.77
126.89
122.12
130.21
115.52
127.39
135.03
139.02
157.28
179.23
173.26
159.75
168.06
168.44
168.1
168.17
181.35
183.99
162.73
143.38
154.05
186.54
194.98
195.71
160.08
119.82
112.26
125.29
127.66
136.16
129.77
101.15
110.98
129.24
123.66
131.24
134.69
138.32
147.3
153.58
142.92
134.74
146.74
139.32
127.09
135.95
144.49
159.91
174.07
194.28
181.26
174.98
185.23
196.2
196.21
198.38
196.47
199.35
199.44
174.65
157.0
167.51
164.57
145.84
144.62
165.2
152.12
159.1
152.65
125.77
115.2
126.57
131.21
131.68
140.07
147.24
151.81
147.28

In [16]:
# sess = tf.Session()
# sess.run(init)
# import gym
# env = gym.make('CartPole-v0')
env.reset()
observation = env.reset()
done = False
t = 0
while t<900:
    env.render()
#     print(observation)
    action = np.argmax(sess.run(myAgent.output,feed_dict={myAgent.state_in:[observation]}))
    observation, reward, done, info = env.step(action)
    t += 1
    if done and False:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.render(close=True)